In [1]:
import pandas as pd
from flask import jsonify
import numpy as np
import requests
import os

In [3]:
df = pd.read_csv('scikit-learn-prs.csv')
df.head()

,additions,author,bodyText,changedFiles,closedAt,comments,commits,createdAt,deletions,id,participants,reactions,state,title
0,153,{'login': 'agramfort'},NaN,7,2010-09-02T14:10:16Z,{'totalCount': 2},{'totalCount': 9},2010-09-01T13:06:00Z,77,MDExOlB1bGxSZXF1ZXN0NjU0,{'totalCount': 3},{'totalCount': 0},MERGED,more on LARS (we're getting there)
1,51,{'login': 'agramfort'},this is just a draft but maybe a good start to...,2,2010-10-23T15:44:33Z,{'totalCount': 1},{'totalCount': 1},2010-09-02T17:13:55Z,0,MDExOlB1bGxSZXF1ZXN0MTEzOQ==,{'totalCount': 4},{'totalCount': 0},CLOSED,Scaling and preprocessing
2,6631,{'login': 'ogrisel'},Implementation of CD for elastic net on scipy....,13,2010-09-14T13:00:43Z,{'totalCount': 2},{'totalCount': 25},2010-09-09T16:03:53Z,9,MDExOlB1bGxSZXF1ZXN0MzEzNg==,{'totalCount': 1},{'totalCount': 0},MERGED,Issue 77 sparse cd
3,105,{'login': 'GaelVaroquaux'},Code review: I am interested in suggestions on...,6,2010-09-16T21:39:45Z,{'totalCount': 1},{'totalCount': 3},2010-09-15T15:48:15Z,27,MDExOlB1bGxSZXF1ZXN0NTA0Nw==,{'totalCount': 2},{'totalCount': 0},MERGED,Work on cross val and pipelines.
4,12636,{'login': 'ogrisel'},NaN,14,2010-10-25T00:27:02Z,{'totalCount': 1},{'totalCount': 17},2010-10-24T15:29:53Z,8,MDExOlB1bGxSZXF1ZXN0MTk0NjY=,{'totalCount': 2},{'totalCount': 0},CLOSED,Some cleanups


In [7]:
def cleanCounts(df):
    '''Remove {totalCount: } from the counts columns to the data is cleaner
       and {login: } from author column'''
    df['author'] = df['author'].str.replace("{'login':", '')
    for i in df.columns:
        if df[i].dtypes == 'O':
            df[i] = df[i].str.replace("{'totalCount':", '')
            df[i] = df[i].str.replace('}', '')
    return df

def findTimeToClose(df):
    '''Generate feature showing the time it take from pull request opening
       to pull request being either merged or closed.
       Account for open prs by using current time for closedAt'''
    df['closedAt'] = pd.to_datetime(df['closedAt'], infer_datetime_format=True)
    df['createdAt'] = pd.to_datetime(df['createdAt'], infer_datetime_format=True)
    df['timeToClosure'] = df['closedAt'] - df['createdAt']
    return df

In [5]:
days = []
for i in df.index:
    if pd.isna(df['closedAt'][i]):
        days.append(pd.Timestamp.now() - df['createdAt'][i])
    else:
        days.append(df['closedAt'][i] - df['createdAt'][i])

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [68]:
df['time_to_closure2'] = pd.Series(days)

In [70]:
df['time_to_closure2'] = df['time_to_closure2'].dt.days
df.head()

,additions,author,bodyText,changedFiles,closedAt,comments,commits,createdAt,deletions,id,participants,reactions,state,title,time_to_closure,time_to_closure2
0,153,'agramfort',NaN,7,2010-09-02 14:10:16,2,9,2010-09-01 13:06:00,77,MDExOlB1bGxSZXF1ZXN0NjU0,3,0,MERGED,more on LARS (we're getting there),1.0,1
1,51,'agramfort',this is just a draft but maybe a good start to...,2,2010-10-23 15:44:33,1,1,2010-09-02 17:13:55,0,MDExOlB1bGxSZXF1ZXN0MTEzOQ==,4,0,CLOSED,Scaling and preprocessing,50.0,50
2,6631,'ogrisel',Implementation of CD for elastic net on scipy....,13,2010-09-14 13:00:43,2,25,2010-09-09 16:03:53,9,MDExOlB1bGxSZXF1ZXN0MzEzNg==,1,0,MERGED,Issue 77 sparse cd,4.0,4
3,105,'GaelVaroquaux',Code review: I am interested in suggestions on...,6,2010-09-16 21:39:45,1,3,2010-09-15 15:48:15,27,MDExOlB1bGxSZXF1ZXN0NTA0Nw==,2,0,MERGED,Work on cross val and pipelines.,1.0,1
4,12636,'ogrisel',NaN,14,2010-10-25 00:27:02,1,17,2010-10-24 15:29:53,8,MDExOlB1bGxSZXF1ZXN0MTk0NjY=,2,0,CLOSED,Some cleanups,0.0,0


In [8]:
df = cleanCounts(df)
df = findTimeToClose(df)
df.head()

,additions,author,bodyText,changedFiles,closedAt,comments,commits,createdAt,deletions,id,participants,reactions,state,title,timeToClosure
0,153,'agramfort',NaN,7,2010-09-02 14:10:16,2,9,2010-09-01 13:06:00,77,MDExOlB1bGxSZXF1ZXN0NjU0,3,0,MERGED,more on LARS (we're getting there),1 days 01:04:16
1,51,'agramfort',this is just a draft but maybe a good start to...,2,2010-10-23 15:44:33,1,1,2010-09-02 17:13:55,0,MDExOlB1bGxSZXF1ZXN0MTEzOQ==,4,0,CLOSED,Scaling and preprocessing,50 days 22:30:38
2,6631,'ogrisel',Implementation of CD for elastic net on scipy....,13,2010-09-14 13:00:43,2,25,2010-09-09 16:03:53,9,MDExOlB1bGxSZXF1ZXN0MzEzNg==,1,0,MERGED,Issue 77 sparse cd,4 days 20:56:50
3,105,'GaelVaroquaux',Code review: I am interested in suggestions on...,6,2010-09-16 21:39:45,1,3,2010-09-15 15:48:15,27,MDExOlB1bGxSZXF1ZXN0NTA0Nw==,2,0,MERGED,Work on cross val and pipelines.,1 days 05:51:30
4,12636,'ogrisel',NaN,14,2010-10-25 00:27:02,1,17,2010-10-24 15:29:53,8,MDExOlB1bGxSZXF1ZXN0MTk0NjY=,2,0,CLOSED,Some cleanups,0 days 08:57:09


In [9]:
df.tail()

,additions,author,bodyText,changedFiles,closedAt,comments,commits,createdAt,deletions,id,participants,reactions,state,title,timeToClosure
9336,38,'glemaitre',similar to #16500 but for plot_precision_recal...,3,NaT,1,2,2020-02-20 22:11:41,5,MDExOlB1bGxSZXF1ZXN0Mzc4MDAxMDQw,1,0,OPEN,BUG ensure that name is properly stored in the...,NaT
9337,164,'rth',Continues and closes #15015\nThis merges the c...,10,NaT,1,10,2020-02-20 23:14:09,15,MDExOlB1bGxSZXF1ZXN0Mzc4MDI0MDQ0,3,0,OPEN,Common check for sample weight invariance with...,NaT
9338,68,'thomasjpfan',Reference Issues/PRs\n\nResolves #16498\nWhat ...,3,NaT,0,2,2020-02-21 00:12:40,0,MDExOlB1bGxSZXF1ZXN0Mzc4MDQxMjQz,1,1,OPEN,ENH Adds pandas IntegerArray support to check_...,NaT
9339,2,'thomasjpfan',Reference Issues/PRs\n\nFixes #16506\nWhat doe...,1,NaT,0,1,2020-02-21 02:26:00,1,MDExOlB1bGxSZXF1ZXN0Mzc4MDcxODQ2,1,0,OPEN,TST Sets random state in test_csr_row_norms,NaT
9340,41,'thomasjpfan',Reference Issues/PRs\n\nContinues #16502\nWhat...,3,NaT,0,2,2020-02-21 03:08:12,8,MDExOlB1bGxSZXF1ZXN0Mzc4MDgwNjIz,1,0,OPEN,TST Checks can now skip test based on estimato...,NaT


In [10]:
df['time_to_closure'] = df['time_to_closure'].dt.days

KeyError: 'time_to_closure'

In [11]:
df.head()

,additions,author,bodyText,changedFiles,closedAt,comments,commits,createdAt,deletions,id,participants,reactions,state,title,timeToClosure
0,153,'agramfort',NaN,7,2010-09-02 14:10:16,2,9,2010-09-01 13:06:00,77,MDExOlB1bGxSZXF1ZXN0NjU0,3,0,MERGED,more on LARS (we're getting there),1 days 01:04:16
1,51,'agramfort',this is just a draft but maybe a good start to...,2,2010-10-23 15:44:33,1,1,2010-09-02 17:13:55,0,MDExOlB1bGxSZXF1ZXN0MTEzOQ==,4,0,CLOSED,Scaling and preprocessing,50 days 22:30:38
2,6631,'ogrisel',Implementation of CD for elastic net on scipy....,13,2010-09-14 13:00:43,2,25,2010-09-09 16:03:53,9,MDExOlB1bGxSZXF1ZXN0MzEzNg==,1,0,MERGED,Issue 77 sparse cd,4 days 20:56:50
3,105,'GaelVaroquaux',Code review: I am interested in suggestions on...,6,2010-09-16 21:39:45,1,3,2010-09-15 15:48:15,27,MDExOlB1bGxSZXF1ZXN0NTA0Nw==,2,0,MERGED,Work on cross val and pipelines.,1 days 05:51:30
4,12636,'ogrisel',NaN,14,2010-10-25 00:27:02,1,17,2010-10-24 15:29:53,8,MDExOlB1bGxSZXF1ZXN0MTk0NjY=,2,0,CLOSED,Some cleanups,0 days 08:57:09


In [44]:
currentTime = pd.Timestamp.now()
currentTime

Timestamp('2020-02-24 10:41:36.926258')

In [48]:
closedAt = df['createdAt'][9340] - currentTime
closedAt

Timedelta('-4 days +16:26:35.073742')

In [8]:
df['time_to_closure'].head()

0    1 days 01:04:16
1   50 days 22:30:38
2    4 days 20:56:50
3    1 days 05:51:30
4    0 days 08:57:09
Name: time_to_closure, dtype: timedelta64[ns]

In [14]:
df['repoName'] = 'scikit-learn'

In [15]:
df['repoName'].head()

0    scikit-learn
1    scikit-learn
2    scikit-learn
3    scikit-learn
4    scikit-learn
Name: repoName, dtype: object

In [2]:
SECRET = os.environ.get('SECRET')
URL = 'https://github.com/scikit-learn/scikit-learn'
DATE_FORMAT = '%Y-%m-%dT%H:%M:%SZ'

In [12]:
def repository():
    user = 'scikit-learn'
    repo = 'scikit-learn'
    query = '''
    query ($user: String!, $repo: String!){
        repository(owner: $user, name: $repo) {
                name
                owner {
                    login
                    }
                description
                primaryLanguage {
                    name
                    }
                stars: stargazers {
                    totalCount
                    }
                forks: forkCount
                totalIssues: issues {
                    totalCount
                    }
                openIssues: issues (states: [OPEN]) {
                    totalCount
                    }
                closedIssues: issues (states: [CLOSED]) {
                    totalCount
                    }
                vulnerabilityAlerts {
                    totalCount
                    }
                totalPRs: pullRequests {
                    totalCount
                    }
                openPRs: pullRequests (states: [OPEN]) {
                    totalCount
                    }
                mergedPRs: pullRequests (states: [MERGED]) {
                    totalCount
                    }
                closedPRs: pullRequests (states: [CLOSED]) {
                    totalCount
                    }
                createdAt
                updatedAt
                diskUsage
                pullRequests (last: 50) {
                nodes {
                    author {
                        login
                        }
                    state
                    createdAt
                    closedAt
                    changedFiles
                    additions
                    deletions
                    }
                }
            }
        }
        '''

    variables = {'user': user, 'repo': repo}
    data = runQuery(query, variables).json()['data']['repository']
    data['stars'] = data['stars']['totalCount']
    data['owner'] = data['owner']['login']
    data['primaryLanguage'] = data['primaryLanguage']['name']
    data['totalIssues'] = data['totalIssues']['totalCount']
    data['openIssues'] = data['openIssues']['totalCount']
    data['closedIssues'] = data['closedIssues']['totalCount']
    data['totalPRs'] = data['totalPRs']['totalCount']
    data['openPRs'] = data['openPRs']['totalCount']
    data['mergedPRs'] = data['mergedPRs']['totalCount']
    data['closedPRs'] = data['closedPRs']['totalCount']
    data['vulnerabilityAlerts'] = data['vulnerabilityAlerts']['totalCount']

    data['PRacceptanceRate'] = data['mergedPRs'] / (data['mergedPRs'] +
                                                    data['closedPRs'])
    data['createdAt'] = datetime.strptime(data['createdAt'],
                                        DATE_FORMAT)
    data['updatedAt'] = datetime.strptime(data['updatedAt'],
                                        DATE_FORMAT)
    data['ageInDays'] = (datetime.now().date() -
                        data['createdAt'].date()).days
    data['starsPerDay'] = data['stars'] / data['ageInDays']
    data['forksPerDay'] = data['forks'] / data['ageInDays']
    data['PRsPerDay'] = data['totalPRs'] / data['ageInDays']
    data['issuesPerDay'] = data['totalIssues'] / data['ageInDays']

    pull_requests = data['pullRequests']['nodes']
    del data['pullRequests']
    df = pd.DataFrame.from_records(pull_requests)
    df['author'] = [author.get('login') if author is not None else ''
                    for author in df['author']]
    df['createdAt'] = pd.to_datetime(df['createdAt'], format=DATE_FORMAT)
    df['closedAt'] = pd.to_datetime(df['closedAt'], format=DATE_FORMAT)

    data['uniquePRauthors'] = df['author'].nunique()

    return data

In [13]:
def runQuery(query, variables):
    r = requests.post(URL,
                      headers={'Authorization': 'token ' + SECRET, },
                      json={'query': query,
                            'variables': variables
                            })
    return r

In [11]:
data = repository()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [24]:
data = runQuery(query, variables).json()['data']['repository']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
os.environ['SECRET'] = 'd27e436e87c1ddcc8fcade768de9082bacbb6d12'

In [15]:
user = 'scikit-learn'
repo = 'scikit-learn'
query = '''
    query ($user: String!, $repo: String!){
        repository(owner: $user, name: $repo) {
                name
                owner {
                    login
                    }
                description
                primaryLanguage {
                    name
                    }
                stars: stargazers {
                    totalCount
                    }
                forks: forkCount
                totalIssues: issues {
                    totalCount
                    }
                openIssues: issues (states: [OPEN]) {
                    totalCount
                    }
                closedIssues: issues (states: [CLOSED]) {
                    totalCount
                    }
                vulnerabilityAlerts {
                    totalCount
                    }
                totalPRs: pullRequests {
                    totalCount
                    }
                openPRs: pullRequests (states: [OPEN]) {
                    totalCount
                    }
                mergedPRs: pullRequests (states: [MERGED]) {
                    totalCount
                    }
                closedPRs: pullRequests (states: [CLOSED]) {
                    totalCount
                    }
                createdAt
                updatedAt
                diskUsage
                pullRequests (last: 50) {
                nodes {
                    author {
                        login
                        }
                    state
                    createdAt
                    closedAt
                    changedFiles
                    additions
                    deletions
                    }
                }
            }
        }
        '''
variables = {'user': user, 'repo': repo}

In [22]:
help(json)

column corresponding to pos
     |  
     |  Method resolution order:
     |      JSONDecodeError
     |      builtins.ValueError
     |      builtins.Exception
     |      builtins.BaseException
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(self, msg, doc, pos)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  __reduce__(self)
     |      Helper for pickle.
     |  
     |  ----------------------------------------------------------------------
     |  Data descriptors defined here:
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)
     |  
     |  ----------------------------------------------------------------------
     |  Static methods inherited from builtins.ValueError:
     |  
     |  __new__(*args, **kwargs) from builtins.type
     |      Create and return a new object.  See help(type) for accurate signature.
     |  
     |  ----------------------

In [23]:
json.

AttributeError: module 'json' has no attribute 'agent'

In [19]:
import json